In [141]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [202]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.summarize import *
from relation_extraction.data.converters.converter_i2b2 import relation_dict
output_path = '/scratch/geeticka/i2b2-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/original/test_original.txt')
entity_blinded_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/entity_blinding/test_entity_blinding.txt')
relation_full_form_dictionary = {'None': 'None', 'PIP': 'PIP', 'TeCP': 'TeCP',
                                 'TeRP': 'TeRP', 'TrAP': 'TrAP', 'TrCP': 'TrCP',
                                 'TrIP': 'TrIP', 'TrNAP': 'TrNAP', 'TrWP': 'TrWP'}
relation_as_short_list = ['None', 'PIP', 'TeCP', 'TeRP', 'TrAP', 'TrCP', 'TrIP', 'TrNAP', 'TrWP']

In [203]:
relation_dict

{0: 'TrIP',
 1: 'TrWP',
 2: 'TrCP',
 3: 'TrAP',
 4: 'TrNAP',
 5: 'TeRP',
 6: 'TeCP',
 7: 'PIP',
 8: 'None'}

In [204]:
confusion_matrix_portion = '<<< 8-WAY EVALUATION >>>'

In [205]:
summary_cm_baseline, summary_cm_conceptblind = print_full_summary('i2b2',
    'result-baseline-i2b2.txt', 'result-concblind-i2b2.txt', 
    'Baseline','Concept-Blind', res, relation_full_form_dictionary, relation_as_short_list, confusion_matrix_portion, 2)


TTest from Baseline to Concept-Blind
Below is the metric comparsion across the two models considering individual relations, excluding 'Other'
Metric: Precision 	 statistic -1.31 	 p_value 0.23216179409004392
Metric: Recall 	 statistic -2.82 	 p_value 0.025707389498527588
Metric: F1 	 statistic -4.02 	 p_value 0.005033639352339186

TTest from Baseline to Concept-Blind
Below is the metric comparsion across the two models considering individual relations, including 'Other'
Metric: Precision 	 statistic -1.37 	 p_value 0.2065001938836207
Metric: Recall 	 statistic -2.65 	 p_value 0.02912898734569872
Metric: F1 	 statistic -3.72 	 p_value 0.005837614083081156

All the macro and micro metrics are calculated, excluding the 'Other' class
Macro - Micro for the Baseline model
Metric: Precision 	 Macro-Micro -12.15
Metric: Recall 	 Macro-Micro -23.99
Metric: F1 	 Macro-Micro -23.31

Macro - Micro for the Concept-Blind model
Metric: Precision 	 Macro-Micro -15.55
Metric: Recall 	 Macro-Micro -27.

In [206]:
summary_cm_baseline

summary,Gold Relation,Confused With(num_examples),Correct Predictions
0,None,PIP(333) TeCP(57) TeRP(321) TrAP(426) TrCP(35)...,11377.0
1,PIP,None(626) TeCP(2) TeRP(10) TrAP(21) TrCP(6) Tr...,694.0
2,TeCP,None(114) PIP(3) TeRP(74) TrAP(9) TrIP(38),100.0
3,TeRP,None(384) PIP(23) TeCP(49) TrAP(13) TrCP(6) Tr...,1509.0
4,TrAP,None(423) PIP(12) TeCP(5) TeRP(5) TrCP(14) TrI...,1169.0
5,TrCP,None(112) PIP(16) TeRP(6) TrAP(56) TrIP(35) Tr...,116.0
6,TrIP,None(35) TeRP(1) TrAP(66) TrCP(5),45.0
7,TrNAP,None(34) PIP(3) TrAP(36) TrCP(4) TrIP(12),23.0
8,TrWP,None(44) PIP(3) TeRP(1) TrAP(38) TrCP(3) TrIP(14),6.0


In [207]:
summary_cm_conceptblind

summary,Gold Relation,Confused With(num_examples),Correct Predictions
0,None,PIP(344) TeCP(46) TeRP(429) TrAP(557) TrCP(54)...,11338.0
1,PIP,None(464) TrIP(1),983.0
2,TeCP,None(98) TeRP(104) TrAP(1) TrIP(7),128.0
3,TeRP,None(315) TeCP(41) TrIP(2),1702.0
4,TrAP,None(395) TrCP(30) TrIP(21) TrNAP(7) TrWP(4),1275.0
5,TrCP,None(106) TrAP(67) TrIP(8) TrNAP(4),157.0
6,TrIP,None(55) TrAP(50) TrCP(9) TrWP(1),37.0
7,TrNAP,None(28) TrAP(46) TrCP(12) TrIP(1),25.0
8,TrWP,None(43) TrAP(35) TrCP(11) TrIP(6) TrNAP(1),13.0


In [251]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [252]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [253]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [254]:
needed_linenum_and_relation = {}

In [255]:
with open(res('answers_for_dev-baseline.txt')) as textfile1, open(res("answers_for_dev-concblind.txt")) as textfile2, \
open(res('answers_for_dev_gold-baseline.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, baseline_relation = read_answers_line(x)
        _, entity_blind_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if entity_blind_relation == gold_relation and baseline_relation != gold_relation:
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, entity_blind_relation)

In [257]:
len(list(needed_linenum_and_relation.keys()))

1815

In [256]:
print('We print the baseline first and then the entity blinded version. Goal relation corresponds to entity blinded\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(entity_blinded_sentences_path) as entity_blinded_sentences:
    for x, y in zip(original_sentences, entity_blinded_sentences):
        needed_linenums = list(needed_linenum_and_relation.keys())
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \t {4}, {5} \nSentences: \n\t{6} \n\t {7}'.format(
                    r_b, r_c, e1_b, e1_c, e2_b, e2_c, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the entity blinded version. Goal relation corresponds to entity blinded


Predicted Relation: 	 None, TeRP 
Entities: 	 Cardiac catheterization, TEST 	 severely diseased LAD, PROBLEM 
Sentences: 
	Cardiac catheterization demonstrates severe triple vessel coronary artery disease , RCA 100% occluded , saphenous vein graft to OM1 graft was 100% occluded and a patent LIMA , but severely diseased LAD . 
	 TEST demonstrates PROBLEM , PROBLEM , TREATMENT to TREATMENT was PROBLEM and a patent LIMA , but PROBLEM .


Predicted Relation: 	 None, TeRP 
Entities: 	 Cardiac catheterization, TEST 	 100% occluded, PROBLEM 
Sentences: 
	Cardiac catheterization demonstrates severe triple vessel coronary artery disease , RCA 100% occluded , saphenous vein graft to OM1 graft was 100% occluded and a patent LIMA , but severely diseased LAD . 
	 TEST demonstrates PROBLEM , PROBLEM , TREATMENT to TREATMENT was PROBLEM and a patent LIMA , but PROBLEM .


Predicted Relation:

	 abd soft , PROBLEM / PROBLEM , NABS. ext wwp , no PROBLEM / PROBLEM , TEST &apos;s b / l.


Predicted Relation: 	 TrAP, TrIP 
Entities: 	 phototherapy, TREATMENT 	 Exaggerated physiologic hyperbilirubinemia, PROBLEM 
Sentences: 
	2. Exaggerated physiologic hyperbilirubinemia responsive to phototherapy . 
	 2. PROBLEM responsive to TREATMENT .


Predicted Relation: 	 PIP, None 
Entities: 	 a stroke, PROBLEM 	 hypertension, PROBLEM 
Sentences: 
	Is significant for a father who had a stroke and also suffered from hypertension . 
	 Is significant for a father who had PROBLEM and also suffered from PROBLEM .


Predicted Relation: 	 TrIP, PIP 
Entities: 	 shower R on T VPB, PROBLEM 	 monomorphic VT, PROBLEM 
Sentences: 
	ECG is AV paced but prior shower R on T VPB c monomorphic VT then overdrive paced . 
	 TEST is AV paced but prior PROBLEM c PROBLEM then overdrive paced .


Predicted Relation: 	 None, TeRP 
Entities: 	 telemetry, TEST 	 further recurrence, PROBLEM 
Sentences: 
	He had occ



Predicted Relation: 	 None, PIP 
Entities: 	 decreased hematocrit, PROBLEM 	 myelodysplastic syndrome, PROBLEM 
Sentences: 
	The patient is a 69-year-old male with multiple medical problems and known transfusion dependent secondary to myelodysplastic syndrome admitted with decreased hematocrit . 
	 The patient is a 69-year-old male with PROBLEM and PROBLEM secondary to PROBLEM admitted with PROBLEM .


Predicted Relation: 	 None, PIP 
Entities: 	 decreased hematocrit, PROBLEM 	 known transfusion dependent, PROBLEM 
Sentences: 
	The patient is a 69-year-old male with multiple medical problems and known transfusion dependent secondary to myelodysplastic syndrome admitted with decreased hematocrit . 
	 The patient is a 69-year-old male with PROBLEM and PROBLEM secondary to PROBLEM admitted with PROBLEM .


Predicted Relation: 	 None, PIP 
Entities: 	 DILATED CARDIOMYOPATHY, PROBLEM 	 CONGESTIVE HEART FAILURE, PROBLEM 
Sentences: 
	5. CONGESTIVE HEART FAILURE WITH DILATED CARDIOMYOPATHY 

	 PROBLEM , PROBLEM .


Predicted Relation: 	 PIP, None 
Entities: 	 multifocal pneumonia, PROBLEM 	 enterobacter, PROBLEM 
Sentences: 
	He was given D50 , but continued to have progressive respiratory failure , was therefore intubated and transferred to the Austen Riggs Center where he was noted to have multifocal pneumonia and bacteremia with enterobacter and MSSA . 
	 He was given TREATMENT , but continued to have PROBLEM , was therefore intubated and transferred to the Austen Riggs Center where he was noted to have PROBLEM and PROBLEM with PROBLEM and PROBLEM .


Predicted Relation: 	 PIP, None 
Entities: 	 Extensive bilateral diffuse ground glass, PROBLEM 	 consolidation in both lungs, PROBLEM 
Sentences: 
	1. Extensive bilateral diffuse ground glass and more confluent areas of consolidation in both lungs consistent with the recent chest x-ray . 
	 1. PROBLEM and more confluent areas of PROBLEM consistent with TEST .


Predicted Relation: 	 TeRP, None 
Entities: 	 Pt 's triple lum

	 His diagnosis of PROBLEM was placed under question on TEST since he was able to have excellent superior gaze especially with TEST which would not be seen with PROBLEM .


Predicted Relation: 	 TrIP, None 
Entities: 	 1+ albumin,, PROBLEM 	 0-5 high link caths, PROBLEM 
Sentences: 
	sodium 141 , potassium 3.5 , chloride 107 , bicarbonates 23.8 , BUN 23 , creatinine 1.1 , glucose 165 , PO2 377 , PCO2 32 , PH 7.50 , asomus 298 , toxic screen negative , white blood cell count 11.1 , hematocrit 39.6 , platelet count 137 , prothrombin time 25.2 , INR 4.3 , partial thromboplastin time 34.7 , urinalysis 1+ albumin, 0-5 high link caths , cervical spine negative , pelvis negative , lumbar spine ; negative , thoracic spine negative . 
	 TEST 141 , TEST 3.5 , TEST 107 , TEST 23.8 , TEST 23 , TEST 1.1 , TEST 165 , TEST 377 , TEST 32 , TEST 7.50 , TEST 298 , TEST negative , TEST 11.1 , TEST 39.6 , TEST 137 , TEST 25.2 , TEST 4.3 , TEST 34.7 , TEST PROBLEM PROBLEM , TEST negative , TEST negative , 

Predicted Relation: 	 TrIP, None 
Entities: 	 a TIBC, TEST 	 Anemia, PROBLEM 
Sentences: 
	Anemia , with a baseline hematocrit of 31.0 , with a TIBC that was low and a high ferritin . 
	 PROBLEM , with TEST of 31.0 , with TEST that was PROBLEM and PROBLEM .


Predicted Relation: 	 TrWP, None 
Entities: 	 a net fluid balance, TEST 	 evidence of bleeding, PROBLEM 
Sentences: 
	The Intensive Care Unit course was also notable for negative chest x-ray , two units of packed red blood cells for a hematocrit of 24.0 with appropriate bump in her hematocrit and no evidence of bleeding , stable blood pressure despite a net fluid balance of negative 2.5 liters . 
	 The Intensive Care Unit course was also notable for negative chest x-ray , TREATMENT for TEST of 24.0 with appropriate bump in TEST and no PROBLEM , stable TEST despite TEST of negative 2.5 liters .


Predicted Relation: 	 TeRP, None 
Entities: 	 blood pressure, TEST 	 evidence of bleeding, PROBLEM 
Sentences: 
	The Intensive Care Unit 

Predicted Relation: 	 TrAP, None 
Entities: 	 ligation, TREATMENT 	 serosal tear of transverse colon, PROBLEM 
Sentences: 
	1. 7-2-93 , emergency room exploratory laparotomy , repair of liver laceration , repair of torn transverse mesocolon , repair of serosal tear of transverse colon , ligation and division of ruptured left renal vein , exploration of pancreas , 
	 1. 7-2-93 , emergency room TEST , TREATMENT of PROBLEM , TREATMENT of PROBLEM , TREATMENT of PROBLEM , TREATMENT and TREATMENT of PROBLEM , TEST ,


Predicted Relation: 	 TrAP, None 
Entities: 	 repair, TREATMENT 	 ruptured left renal vein, PROBLEM 
Sentences: 
	1. 7-2-93 , emergency room exploratory laparotomy , repair of liver laceration , repair of torn transverse mesocolon , repair of serosal tear of transverse colon , ligation and division of ruptured left renal vein , exploration of pancreas , 
	 1. 7-2-93 , emergency room TEST , TREATMENT of PROBLEM , TREATMENT of PROBLEM , TREATMENT of PROBLEM , TREATMENT and TREATM